In [ ]:
# import matplotlib for when it's necessary to plot images, graphics, figures, etc.
import matplotlib
matplotlib.use("Agg")
 
# change enviroment path
import os

# format output data for Keras classification
from keras.utils import to_categorical

# necessary imports for managing Keras models
from tensorflow.keras.models import Sequential, Model, model_from_json

# layers type that can be used for block the network
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Activation, Dropout

# transfer learning architectures
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, InceptionV3, MobileNet, ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121

# data generator for data augmentation
from keras.preprocessing.image import ImageDataGenerator

# network optimizer
from tensorflow.keras.optimizers import Adam

# model validation
from sklearn.model_selection import StratifiedKFold

# managing images
from tensorflow.keras.preprocessing.image import img_to_array
from imutils import paths
import cv2
import random
from tensorflow.keras.preprocessing import image

# tensorflow
import tensorflow as tf
from tensorflow.keras import backend as K

# numpy 
import numpy as np

# metrics
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# change folder where is the data
os.chdir("../database/")

# general parameters
EPOCHS = 50
BS = 32

# image input for resizing
HEIGHT = 256
WIDTH = 256
INPUT_SHAPE = (WIDTH,HEIGHT,3)

In [ ]:
# this function get the data for training and classification
# this returns: array of images, array of their labels
def get_data(path_data):
    # initialize the data and labels
    print("[INFO] loading images...")
    data = []
    labels = []
     
    # grab the image paths and randomly shuffle them
    imagePaths = sorted(list(paths.list_images(path_data)))
    random.seed(42)
    random.shuffle(imagePaths)

    # loop over the input images
    for imagePath in imagePaths:
        
        # read the image
        image = cv2.imread(imagePath)
        # resize it
        image = cv2.resize(image, (HEIGHT,WIDTH)) 
        image = img_to_array(image)
        
        #if working on linux, use the following line
        label = imagePath.split("/")[-2]
        
        #if working on windows, use the following line
        #label = imagePath.split("\\")[-2]
        
        
        #FOR MULTICLASS CLASSIFICATION USE THE CODE BELLOW
        if label == "NORMAL":
            label = 0
            # adding the image into array
            data.append(image)
            labels.append(label)
        elif label == "COVID-19":
            label = 1
            data.append(image)
            labels.append(label)
        elif label == "VIRAL":
            label = 2
            data.append(image)
            labels.append(label)
        
        '''     
        #FOR VIRAL VS COVID USE THE CODE BELLOW
        if label == "VIRAL":
            label = 0
            data.append(image)
            labels.append(label)
        elif label == "COVID-19":
            label = 1
            data.append(image)
            labels.append(label)
        '''
        '''
        #FOR NORMAL VS COVID USE THE CODE BELLOW
        if label == "NORMAL":
            label = 0
            data.append(image)
            labels.append(label)
        elif label == "COVID-19":
            label = 1
            data.append(image)
            labels.append(label)
        '''
        

    # scale the raw pixel intensities to the range [0, 1]
    data = np.array(data, dtype="float16") / 255.0
    labels = np.array(labels)

    return data, labels

In [ ]:
# this function defines the transfer learning model based on vgg16 + 2 dense layers
def get_vgg16_model():
    conv_base = VGG16(input_shape=INPUT_SHAPE, include_top = False, weights='imagenet')

    #freezing the weights of the original vgg16 model
    for layer in conv_base.layers:
        layer.trainable = False

    x = conv_base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x) 
    x = Dense(256, activation='relu')(x) 
    
    # use this line bellow for binary classification
    predictions = Dense(2, activation='sigmoid')(x)
    # use this line bellow for multiclass classification
    #predictions = Dense(3, activation='sigmoid')(x)
    modelVGG16 = Model(conv_base.input, predictions)
    
    return modelVGG16

In [ ]:
# this function defines the transfer learning model based on vgg19 + 2 dense layers
def get_vgg19_model():
    conv_base = VGG19(input_shape=INPUT_SHAPE, include_top = False, weights='imagenet')

    for layer in conv_base.layers:
        layer.trainable = False

    x = conv_base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x) 
    x = Dense(256, activation='relu')(x) 
    
    # use this line bellow for binary classification
    predictions = Dense(2, activation='sigmoid')(x)
    # use this line bellow for multiclass classification
    #predictions = Dense(3, activation='sigmoid')(x)
    modelVGG19 = Model(conv_base.input, predictions)
    
    return modelVGG19

In [ ]:
# this function defines the transfer learning model based on mobilenet + 2 dense layers
def get_mobilenet_model():

    conv_base = MobileNet(
        include_top=False,
        weights='imagenet')

    #freezing the weights
    for layer in conv_base.layers:
        layer.trainable = False

    x = conv_base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x) 
    x = Dense(256, activation='relu')(x) 
    
    # use this line bellow for binary classification
    predictions = Dense(2, activation='sigmoid')(x)
    # use this line bellow for multiclass classification
    #predictions = Dense(3, activation='sigmoid')(x)
    modelMobileNet = Model(conv_base.input, predictions)

    return modelMobileNet

In [ ]:
# this function defines the transfer learning model based on resnet50 + 2 dense layers
def get_resnet50_model():

    conv_base = ResNet50(
        include_top=False,
        weights='imagenet')

    #freezing the weights
    for layer in conv_base.layers:
        layer.trainable = False

    x = conv_base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x) 
    x = Dense(256, activation='relu')(x) 
    
    # use this line bellow for binary classification
    predictions = Dense(2, activation='sigmoid')(x)
    # use this line bellow for multiclass classification
    #predictions = Dense(3, activation='sigmoid')(x)
    modelMobileNet = Model(conv_base.input, predictions)

    return modelMobileNet

In [ ]:
# this function defines the transfer learning model based on inceptionv3 + 2 dense layers
def get_InceptionV3_model():

    conv_base = InceptionV3(
        include_top=False,
        weights='imagenet')

    #freezing the weights
    for layer in conv_base.layers:
        layer.trainable = False

    x = conv_base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x) 
    x = Dense(256, activation='relu')(x) 
    
    # use this line bellow for binary classification
    predictions = Dense(2, activation='sigmoid')(x)
    # use this line bellow for multiclass classification
    #predictions = Dense(3, activation='sigmoid')(x)
    modelMobileNet = Model(conv_base.input, predictions)

    return modelMobileNet

In [ ]:
# this function defines the transfer learning model based on densenet + 2 dense layers
def get_DenseNet121_model():

    conv_base = DenseNet121(
        include_top=False,
        weights='imagenet')

    #freezing the weights
    for layer in conv_base.layers:
        layer.trainable = False

    x = conv_base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x) 
    x = Dense(256, activation='relu')(x) 
    
    # use this line bellow for binary classification
    predictions = Dense(2, activation='sigmoid')(x)
    # use this line bellow for multiclass classification
    #predictions = Dense(3, activation='sigmoid')(x)
    modelMobileNet = Model(conv_base.input, predictions)

    return modelMobileNet

In [ ]:
# code for saving the model for each fold of cross validation
# the model name will be saved with namemodel + step fold validation
def save_model(name, model):

    model_json = model.to_json()
    with open("../Models/"+name+".json", "w") as json_file:
        json_file.write(model_json)
    
    model.save_weights("../Models/"+name".h5")
    print("Saved model "+name+" to disk")

In [ ]:
# this function makes the cross validation training
def cross_validation(X,y,model_type,k_folds,name):
    print("[INFO] Preparing the fold")
    
    # K=0
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
    kf.get_n_splits(data)
    
    print("[INFO] Starting the cross-validation...")
    i = 1
    for train_index, test_index in kf.split(X,y):
        trainData = X[train_index]
        testData = X[test_index]
        trainLabels = y[train_index]
        testLabels = y[test_index]
        
        trainLabels =to_categorical(trainLabels)
        testLabels =to_categorical(testLabels)

        print("=========================================")
        print("====== K Fold Validation step => %d/%d =======" % (i,10))
        print("=========================================")

        # construct the image generator for data augmentation
        generator = ImageDataGenerator(shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
        
        #############################################################
        model = None
        
        if model_type == "vgg16":
            model = get_vgg16_model()
        elif model_type == "vgg19":
            model = get_vgg19_model()
        elif model_type == "mobilenet":
            model = get_mobilenet_model()
        elif model_type == "resnet50":
            model = get_resnet50_model()
        elif model_type == "inceptionv3":
            model = get_InceptionV3_model()
        elif model_type == "densenet":
            model = get_DenseNet121_model()
        #############################################################
        

        model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

        print("[INFO] training network...")
        
        # training with data augmentation
        history = model.fit_generator(generator.flow(trainData, trainLabels, batch_size=BS),
        validation_data=(testData, testLabels), steps_per_epoch=len(trainData) // BS,
        epochs=50, verbose=1)
        
        
        #############################################################
        save_model(name+"-step-fold-"+str(i),model)
        model = None
        del(model)
        #############################################################
        
        i+=1
        
    return kf

In [ ]:
# gettting the data for experiments
# please inform all way e.g. "C:\\Users\\user\\folder\\etc..." 
data, labels = get_data("/home/hosa/ownCloud/dotLab/experimental/database/")

X = data
y = labels

K = 10

In [ ]:
####Execution of VGG-16
name = "VGG16-multi9lass"
folds_vgg19 = cross_validation(X,y,"vgg16",K,name)

In [ ]:
####Execution of VGG-19
name = "VGG19-multiclass"
folds_vgg19 = cross_validation(X,y,"vgg19",K,name)

In [ ]:
####Execution of MOBILENET
name = "mobilenet-multiclass"
folds_cnn3264128 = cross_validation(X,y,"mobilenet",K,name)

In [ ]:
####Execution of INCEPTIONV3
name = "inceptionv3-multiclass"
folds_cnn3264128 = cross_validation(X,y,"inceptionv3",K,name)

In [ ]:
####Execution of DENSENET
name = "densenet-multiclass"
folds_cnn3264128 = cross_validation(X,y,"densenet",K,name)

In [ ]:
####Execution of RESNET50
name = "resnet50-multiclass"
folds_cnn3264128 = cross_validation(X,y,"resnet50",K,name)

In [ ]:
# this function load the saved model to memory
def load_model(name):
    # load json and create model
    json_file = open("../Models/"+name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights("../Models/"+name+".h5")
    
    loaded_model.compile(optimizer='adam', loss='binary_crossentropy', 
                        metrics=['acc'])
                         #metrics=['acc', sensitivity, specificity])
                        

    #print("Loaded model "+name+" from disk")
    
    return loaded_model

In [ ]:
# this function returns the accuracy, precision, recall, and f1-score of classification 
def eval(true, predicted):
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score
    from numpy import argmax, unique
    
    
    true = argmax(true,axis=1)
    #print(unique(true))
    predicted = argmax(predicted,axis=1)
    #print(unique(predicted))
    
    acc = accuracy_score(true,predicted)
    pcs, rec, f1, sup = precision_recall_fscore_support(true, predicted, average='weighted')
    
    print(str(acc*100)+" "+str(pcs*100)+" "+str(rec*100)+" "+str(f1*100))
    

In [ ]:
# this function loads from disk all saved models of a specific technique, then make 10 executions (10 folds) to evaluate the model
def cross_evaluation(X,y,model_type,k_folds,name):
    print("[INFO] Preparing the folds...")
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
    kf.get_n_splits(data)
    
    print("[INFO] Starting the cross-evaluation for ",name+model_type, " ....")
    i = 1
    for train_index, test_index in kf.split(X,y):
        #just consider the test data for evaluation, train data can indicate overfitting
        testData = X[test_index]
        testLabels = y[test_index]
        
        testLabels =to_categorical(testLabels)

        
        model = None
        model = load_model(name+"-step-fold-"+str(i))
        
        pred = model.predict(testData)
        eval(testLabels,pred)
        
        
        #############################################################
        model = None
        del(model)
        #############################################################
        
        i+=1
        
    return kf

# Calculates the results for multiclass classification

## please consider the full way of the model and its name (for multiclass or binary same)

In [ ]:
name = "multiclass/densenet-multiclass"
cross_evaluation(data,y,"densenet",K,name)

In [ ]:
name = "multiclass/inceptionv3-multiclass"
cross_evaluation(data,y,"inceptionv3",K,name)

In [ ]:
name = "multiclass/mobilenet-multiclass"
cross_evaluation(data,y,"mobilenet",K,name)

In [ ]:
name = "multiclass/resnet50-multiclass"
cross_evaluation(data,y,"resnet50",K,name)

In [ ]:
name = "multiclass/VGG16-multiclass"
cross_evaluation(data,y,"VGG16",K,name)

In [ ]:
name = "multiclass/VGG19-multiclass"
cross_evaluation(data,y,"VGG19",K,name)

# Calculates the results for binary COVID vs Normal

In [ ]:
name = "covid19vsnormal/densenet-multiclass"
cross_evaluation(data,y,"densenet",K,name)

In [ ]:
name = "covid19vsnormal/inceptionv3-multiclass"
cross_evaluation(data,y,"inceptionv3",K,name)

In [ ]:
name = "covid19vsnormal/mobilenet-multiclass"
cross_evaluation(data,y,"mobilenet",K,name)

In [ ]:
name = "covid19vsnormal/resnet50-multiclass"
cross_evaluation(data,y,"resnet50",K,name)

In [ ]:
name = "covid19vsnormal/VGG16-multi9lass"
cross_evaluation(data,y,"VGG16",K,name)

In [ ]:
name = "covid19vsnormal/VGG19-multiclass"
cross_evaluation(data,y,"VGG19",K,name)

# Calculates the results for COVID vs Pneum viral

In [ ]:
name = "covid19vsviral/densenet-multiclass"
cross_evaluation(data,y,"densenet",K,name)

In [ ]:
name = "covid19vsviral/inceptionv3-multiclass"
cross_evaluation(data,y,"inceptionv3",K,name)

In [ ]:
name = "covid19vsviral/mobilenet-multiclass"
cross_evaluation(data,y,"mobilenet",K,name)

In [ ]:
name = "covid19vsviral/resnet50-multiclass"
cross_evaluation(data,y,"resnet50",K,name)

In [ ]:
name = "covid19vsviral/VGG16-multiclass"
cross_evaluation(data,y,"VGG16",K,name)

In [ ]:
name = "covid19vsviral/VGG19-multiclass"
cross_evaluation(data,y,"VGG19",K,name)